In [1]:
import os 
import json
from google.cloud import storage

In [2]:
dataset_version = "new_image"

image_folder = f"./dataset/{dataset_version}"
json_folder = f"./json_output/{dataset_version}"

In [3]:
# Initialize GCP
# gcp_cred = os.environ.get("GCP_CRED")
# os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = str(gcp_cred) #"gcp_credentials.json"
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = str(os.environ["GOOGLE_APPLICATION_CREDENTIALS_PATH"])

GCS_BUCKET = os.environ.get("GCS_BUCKET_NAME")
bucket = storage.Client().bucket(GCS_BUCKET)

base_url = "https://storage.googleapis.com"


In [4]:
def upload_to_gcs(source_file_name, destination_blob_name):
    blob = bucket.blob(destination_blob_name)
    blob.upload_from_filename(source_file_name)
    print(f"File {source_file_name} uploaded to {destination_blob_name}.")

In [5]:
def image_exists(folder, filename_wo_ext):
    # Common image extensions (you can add more if needed)
    image_extensions = ['.jpg', '.jpeg', '.png', '.gif', '.bmp', '.tiff', '.webp']

    # Loop through each possible extension
    for ext in image_extensions:
        full_path = os.path.join(folder, filename_wo_ext + ext)
        if os.path.isfile(full_path):
            return full_path  # Return the full path if found

    return False

In [6]:
for json_file in os.listdir(json_folder):
    # print(json_file)
    json_path = f"{json_folder}/{json_file}"
    print(json_path)
    with open(json_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    sku = str(data['SKU'])

    # image_path = f"{image_folder}/{sku}.jpg"
    image_path = image_exists(image_folder, sku)

    if os.path.exists(image_path):

        image_filename = os.path.basename(image_path)
        image_destination_blob_name = 'ai-assistant/textures/'+str(sku)+'/'+str(image_filename)
        upload_to_gcs(source_file_name=image_path, destination_blob_name=image_destination_blob_name)

        image_url = f"{base_url}/{GCS_BUCKET}/{image_destination_blob_name}"
        print(f"image url: {image_url}")
        data["image_path"] = image_url

        with open(json_path, 'w', encoding='utf-8') as f:
            json.dump(data, f, ensure_ascii=False, indent=4)

        print(data)
        json_destination_blob_name = 'ai-assistant/textures/'+str(sku)+'/'+str(json_file)
        upload_to_gcs(source_file_name=json_path, destination_blob_name=json_destination_blob_name)


./json_output/new_image/Z21B1D57ANAVI301.json
File ./dataset/new_image\Z21B1D57ANAVI301.jpg uploaded to ai-assistant/textures/Z21B1D57ANAVI301/Z21B1D57ANAVI301.jpg.
image url: https://storage.googleapis.com/weshape-dev/ai-assistant/textures/Z21B1D57ANAVI301/Z21B1D57ANAVI301.jpg
{'SKU': 'Z21B1D57ANAVI301', 'ID': 'GP_XP_NAVI_RIVERWAY', 'Name': 'GP XP-NAVI RIVERWAY (POL) R/T 60X120PM', 'Thai Name': 'GP 60X120 XP-นาวี ริเวอร์เวย์ (POL)R/TPM', 'Pack': '30 boxes/pallet, 2 pieces/box, 1.44 sq.m./box', 'Texture': 'Polish', 'Price': '711 TH/Unit', 'Unit': 'M2', 'Tile/Plank Width': '40', 'Max Tile/Plank Length': '40', 'Surface_applicability': 'any', 'Collection': 'other', 'description': 'The tile image features a marble-like surface with a blend of gray, white, and subtle yellow veins, creating a natural and elegant appearance. The surface has a smooth texture with a glossy finish, enhancing the depth and richness of the stone pattern./ ภาพกระเบื้องแสดงพื้นผิวคล้ายหินอ่อนที่มีการผสมผสานของสีเทา 